In [ ]:
# Stroop, fluency, and the ACC post hoc tests
# J Ruffle j.ruffle@ucl.ac.uk

In [ ]:
from graph_tool.all import *; import graph_tool.all as gt; import  matplotlib; import math; import numpy; import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
#matplotlib.use('Agg')
import pandas as pd
import scipy as scipy
import seaborn as sns
from pylab import *
from multiprocessing import  Pool
import subprocess
import scipy.sparse as sparse
from scipy.io import savemat
# sns.set_theme()
import pickle
pd.set_option('display.max_columns', 500)
import glob

import nibabel as nib

from nilearn import plotting
from nilearn import surface
from nilearn import datasets

import os
import shutil

from tqdm.notebook import tqdm 

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

sns.set_palette('Set2')

In [ ]:
path = '/home/jruffle/OneDrive/PhD/Stroop_GLDM/11_CORTEX_PAPER_analysis/'
inpath = path+'inputs/'
outpath = path+'results/'

level = 1
debug=True
invert_stroop=True
pca=True

In [ ]:
infile = open(inpath+'state_layered_thresh.p','rb')
state=pickle.load(infile)

In [ ]:
parcellation_nii = nib.load(inpath+'layered_sbm_l'+str(level)+'.nii.gz')
parcellation = np.asanyarray(parcellation_nii.dataobj)

In [ ]:
plotting.plot_roi(parcellation_nii, title="Stochastic block model parcellation: L"+str(level))

In [ ]:
target_error_nii = nib.load(inpath+'layered_sbm_l'+str(level)+'_posterior_mean.nii.gz')
plotting.plot_glass_brain(target_error_nii, title='Stroop reconstruction')

In [ ]:
df = pd.read_excel(inpath+'Stroop_3_05_23.xlsx')
target = 'Stroop_words_correct'

df['Hosp_Number']=''
for i, row in df.iterrows():
    df.loc[i,'Hosp_Number']=row['Original_scan_filename'].split('_')[0].split('.')[0]
df.drop(['Group','Laterality','Original_scan_filename','Lesion_mask_filename'],axis=1,inplace=True)

#add column for each parcel at level
for p in np.unique(parcellation[parcellation>0]):
    df[int(p)]=0

df

In [ ]:
def clamp(train):
    perc01 = np.percentile(train, 5, keepdims=True)
    perc99 = np.percentile(train, 95, keepdims=True)
    clamped_train = np.clip(train, a_min=perc01, a_max=perc99)
    return clamped_train

def zscore(train):
    m = np.mean(train)
    sd = np.std(train)l
    zscored_train = (train-m)/sd
    return zscored_train

def normalise(train):
    normalised_train = (train - np.min(train)) / (np.max(train) - np.min(train))
    return normalised_train

In [ ]:
df['Stroop_words_correct'].min()

In [ ]:
if invert_stroop:
    df['Stroop_words_correct']=1/df['Stroop_words_correct']
    if debug:
        print('Inverting stroop target') 

In [ ]:

df['Stroop_words_correct'] = normalise(zscore(clamp(df['Stroop_words_correct'].values)))
        
sns.histplot(df['Stroop_words_correct'])

In [ ]:
for i, row in df.iterrows():
    print(row['Stroop_words_correct'])

In [ ]:
df.shape

In [ ]:
stroop_nii = nib.load(inpath+'stroop_layered_sbm_l0_posterior_mean_clusterthreshold.nii.gz')
affine = stroop_nii.affine
stroop = np.asanyarray(stroop_nii.dataobj)
stroop[stroop<0.35]=0
stroop[stroop>0]=1
stroop=stroop.astype(int)

cingulate_nii = nib.load(inpath+'stroop_444_thresh_cc_bin.nii.gz')
cingulate = np.asanyarray(cingulate_nii.dataobj)
cingulate=cingulate.astype(int)

In [ ]:
parcellation_stroop = parcellation.copy()
parcellation_cingulate = parcellation.copy()

parcellation_stroop = parcellation_stroop*stroop
parcellation_cingulate = parcellation_cingulate*cingulate

stroop_parcels = np.unique(parcellation_stroop[parcellation_stroop>0]).astype(int)
cingulate_parcels = np.unique(parcellation_cingulate[parcellation_cingulate>0]).astype(int)

if debug:
    print('Stroop parcels: '+str(stroop_parcels))
    print("")
    print('Cingulate parcels: '+str(cingulate_parcels))

In [ ]:
parcellation_cingulate_nii = nib.Nifti1Image(parcellation_cingulate,affine=affine)
parcellation_stroop_nii = nib.Nifti1Image(parcellation_stroop,affine=affine)

nib.save(parcellation_cingulate_nii,outpath+'parcellation_cingulate_nii.nii.gz')
nib.save(parcellation_stroop_nii,outpath+'parcellation_stroop_nii.nii.gz')

In [ ]:
plotting.plot_roi(parcellation_cingulate_nii, title="Cingulate internal parcellation: L"+str(level))

In [ ]:
plotting.plot_roi(parcellation_stroop_nii, title="Stroop internal parcellation: L"+str(level))

In [ ]:
drop_these = []
df_volume = df.copy()
df_prop = df.copy()
worked_counter =0

counts_native = np.unique(parcellation,return_counts=True)
counts_native = pd.DataFrame(counts_native).T
counts_native.rename(columns={0:'parcel',1:'Volume'},inplace=True)
counts_native

lesion_path = '/home/jruffle/OneDrive/PhD/Stroop_GLDM/11_CORTEX_PAPER_analysis/inputs/mni_masks_444/'
for participant, row in tqdm(df.iterrows(),total=len(df)):
    
    sub = row['Hosp_Number']
    lesion = glob.glob(lesion_path+'*'+sub+'*')
    
    if len(lesion)==0:
        drop_these.append(sub)
        df.drop(participant,axis=0,inplace=True)
        df_volume.drop(participant,axis=0,inplace=True)
        df_prop.drop(participant,axis=0,inplace=True)
        
        if debug:
            print('Appending to drop list: '+str(sub))
        
    if len(lesion)>0:
        segmentation_nii = nib.load(lesion[0])
        segmentation = np.asanyarray(segmentation_nii.dataobj)
        segmentation[segmentation>0]=1
        
        parcellation_segmentation = parcellation.copy()
        parcellation_segmentation = parcellation_segmentation*segmentation
        
        if debug:
            print(sub)
            print('Parcels in lesion: '+str(np.unique(parcellation_segmentation[parcellation_segmentation>0])))
        worked_counter +=1
            
        df.loc[participant,np.unique(parcellation_segmentation[parcellation_segmentation>0]).astype(int)]=1
        
        counts = np.unique(parcellation_segmentation,return_counts=True)
        
        for count in range(len(counts[0])):
            if counts[0][count]>0:
                df_volume.loc[participant,int(counts[0][count])]=int(counts[1][count])
                
                #prop
                total_vol = counts_native.loc[counts_native['parcel']==counts[0][count],'Volume'].values[0]
                prop = int(counts[1][count]) / total_vol
                df_prop.loc[participant,int(counts[0][count])]=prop

In [ ]:
df.drop(['Age','Hosp_Number'],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)
df_volume.drop(['Age','Hosp_Number'],axis=1,inplace=True)
df_volume.reset_index(drop=True,inplace=True)
df_prop.drop(['Age','Hosp_Number'],axis=1,inplace=True)
df_prop.reset_index(drop=True,inplace=True)

In [ ]:
def rename_cols(dataframe,pca=pca,mode='l_frontal'):
    for c in dataframe.columns:
        if c !='Stroop_words_correct':
            if pca:
                dataframe.rename(columns={c:mode+'_PC_'+str(c)},inplace=True)
            else:
                dataframe.rename(columns={c:'parcel_'+str(c)},inplace=True)
    dataframe.reset_index(drop=True,inplace=True)
    return dataframe

In [ ]:
if pca:
    from sklearn.decomposition import PCA
    print('Enabling PCA mode')
    comps=5

In [ ]:
def generate_pca(df,comp=5):
    pca = PCA(n_components=comp)
    pca_df = pca.fit_transform(df)
    print(pca.fit(df).explained_variance_ratio_.sum())
    return pca_df

In [ ]:
all_parcels = np.concatenate((stroop_parcels,cingulate_parcels))

Stroop_words_correct = df['Stroop_words_correct']

df = df[all_parcels]
df['Stroop_words_correct']=Stroop_words_correct

df_prop = df_prop[all_parcels]
df_prop['Stroop_words_correct']=Stroop_words_correct

df_volume = df_volume[all_parcels]
df_volume['Stroop_words_correct']=Stroop_words_correct

In [ ]:
df_stroop = df[stroop_parcels]
if pca:
    df_stroop = pd.DataFrame(normalise(generate_pca(df[stroop_parcels])))

df_stroop['Stroop_words_correct']=Stroop_words_correct
df_stroop = rename_cols(df_stroop)
df_stroop.to_csv(outpath+'df_stroop.csv')

df_cingulate = df[cingulate_parcels]
if pca:
    df_cingulate = pd.DataFrame(normalise(generate_pca(df[cingulate_parcels])))
df_cingulate['Stroop_words_correct']=Stroop_words_correct
df_cingulate = rename_cols(df_cingulate,mode='cingulate')
df_cingulate.to_csv(outpath+'df_cingulate.csv')

if pca:
    df = pd.concat([df_stroop,df_cingulate],axis=1)
    df = df.loc[:,~df.columns.duplicated()].copy()
if not pca:
    df = rename_cols(df)
df.to_csv(outpath+'df.csv')

In [ ]:
df_prop_stroop = df_prop[stroop_parcels]
if pca:
    df_prop_stroop = pd.DataFrame(normalise(generate_pca(df_prop[stroop_parcels])))
df_prop_stroop['Stroop_words_correct']=Stroop_words_correct
df_prop_stroop = rename_cols(df_prop_stroop)
df_prop_stroop.to_csv(outpath+'df_prop_stroop.csv')

df_prop_cingulate = df_prop[cingulate_parcels]
if pca:
    df_prop_cingulate = pd.DataFrame(normalise(generate_pca(df_prop[cingulate_parcels])))
df_prop_cingulate['Stroop_words_correct']=Stroop_words_correct
df_prop_cingulate = rename_cols(df_prop_cingulate,mode='cingulate')
df_prop_cingulate.to_csv(outpath+'df_prop_cingulate.csv')

if pca:
    df_prop = pd.concat([df_prop_stroop,df_prop_cingulate],axis=1)
    df_prop = df_prop.loc[:,~df_prop.columns.duplicated()].copy()
if not pca:
    df_prop = rename_cols(df_prop)
df_prop.to_csv(outpath+'df_prop.csv')

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scale = StandardScaler()
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
import sklearn

In [ ]:
df_volume_stroop = df_volume[stroop_parcels]
if pca:
    df_volume_stroop = pd.DataFrame(normalise(generate_pca(df_volume[stroop_parcels])))
df_volume_stroop['Stroop_words_correct']=Stroop_words_correct
df_volume_stroop = rename_cols(df_volume_stroop)
df_volume_stroop.to_csv(outpath+'df_volume_stroop.csv')

df_volume_cingulate = df_volume[cingulate_parcels]
if pca:
    df_volume_cingulate = pd.DataFrame(normalise(generate_pca(df_volume[cingulate_parcels])))
df_volume_cingulate['Stroop_words_correct']=Stroop_words_correct
df_volume_cingulate = rename_cols(df_volume_cingulate,mode='cingulate')
df_volume_cingulate.to_csv(outpath+'df_volume_cingulate.csv')

if pca:
    df_volume = pd.concat([df_volume_stroop,df_volume_cingulate],axis=1)
    df_volume = df_volume.loc[:,~df_volume.columns.duplicated()].copy()
if not pca:
    df_volume = rename_cols(df_volume)
df_volume.to_csv(outpath+'df_volume.csv')

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,5))

sns.heatmap(df_volume.corr(),ax=ax[0])
ax[0].set_title('Volumetric corrmat')

sns.heatmap(df.corr(),ax=ax[1])
ax[1].set_title('Binary corrmat')

sns.heatmap(df_prop.corr(),ax=ax[2])
ax[2].set_title('Proportionate corrmat')

plt.savefig(outpath+"bayesreg_corr",dpi=300,bbox_inches='tight')


In [ ]:
import os 
try:
    os.remove(outpath+'bayes_reg_posteriors.txt')
    print('removing last attempt')
except:
    print('no file to remove')

In [ ]:
print('Running matlab script now')
!matlab -nodisplay -nosplash -nodesktop -r "run('/home/jruffle/OneDrive/PhD/scripts_JKR/fluid_intelligence/newIQ-graph-lesion-deficit-mapping/stroop_bayesreg.m');exit;"

In [ ]:
bayes_reg = pd.read_csv(outpath+'bayes_reg.csv')
priors = ['ridge','g','hs','hs+','lasso']

bayes_reg['Damage formulation'] = ''
bayes_reg['Parcels']=''

for i, row in bayes_reg.iterrows():
    if 'cin' in row['model']:
        bayes_reg.loc[i,'Parcels']='Cingulate'
    if 'str' in row['model']:
        bayes_reg.loc[i,'Parcels']='Left frontal'
    if 'all' in row['model']:
        bayes_reg.loc[i,'Parcels']='All'

    if 'bin' in row['model']:
        bayes_reg.loc[i,'Damage formulation']='Binary'
    if 'vol' in row['model']:
        bayes_reg.loc[i,'Damage formulation']='Volumetric'
    if 'pro' in row['model']:
        bayes_reg.loc[i,'Damage formulation']='Proportionate'

bayes_reg.replace('df_cin_bin','Cingulate parcel (NeuroQuery) (binary impact)',inplace=True)
bayes_reg.replace('df_str_bin','Left frontal stroop parcel (ours) (binary impact)',inplace=True)
bayes_reg.replace('df_all_bin','All parcels (binary impact)',inplace=True)
bayes_reg.replace('df_cin_vol','Cingulate parcel (NeuroQuery) (volumetric impact)',inplace=True)
bayes_reg.replace('df_str_vol','Left frontal stroop parcel (ours) (volumetric impact)',inplace=True)
bayes_reg.replace('df_all_vol','All parcels (volumetric impact)',inplace=True)
bayes_reg.replace('df_cin_pro','Cingulate parcel (NeuroQuery) (proportionate impact)',inplace=True)
bayes_reg.replace('df_str_pro','Left frontal stroop parcel (ours) (proportionate impact)',inplace=True)
bayes_reg.replace('df_all_pro','All parcels (proportionate impact)',inplace=True)

bayes_reg.to_csv(outpath+'bayes_reg_revised_labels.csv')
bayes_reg.sort_values(by='r2')

In [ ]:
bayes_reg.shape

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5))
sns.boxplot(data=bayes_reg,x='Parcels',y='r2',hue='Damage formulation',ax=ax[0],legend=False)
# ax[0].set_title('Bayesian Regression R2')
ax[0].set(ylabel='R2')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=15,ha='right',rotation_mode='anchor')

sns.boxplot(data=bayes_reg,x='Parcels',y='waic',hue='Damage formulation',ax=ax[1])
# ax[1].set_title('Bayesian Regression WAIC')
ax[1].set(ylabel='WAIC')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=15,ha='right',rotation_mode='anchor')

fig.suptitle("Bayesian Regression",y=0.93)

plt.savefig(outpath+"bayesreg_subplot",dpi=300,bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5))
sns.boxplot(data=bayes_reg,hue='Parcels',y='r2',x='Damage formulation',ax=ax[0],legend=False)
# ax[0].set_title('Bayesian Regression R2')
ax[0].set(ylabel='R2')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=15,ha='right',rotation_mode='anchor')

sns.boxplot(data=bayes_reg,hue='Parcels',y='waic',x='Damage formulation',ax=ax[1])
# ax[1].set_title('Bayesian Regression WAIC')
ax[1].set(ylabel='WAIC')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=15,ha='right',rotation_mode='anchor')

fig.suptitle("Bayesian Regression",y=0.93)

plt.savefig(outpath+"bayesreg_subplot_recoloured",dpi=300,bbox_inches='tight')

In [ ]:
filename = '/home/jruffle/OneDrive/PhD/Stroop_GLDM/11_CORTEX_PAPER_analysis/results/bayes_reg_posteriors.txt'
# # filename = "./dataset/sample_data.txt"
# coef = pd.read_csv(filename)

file = open(filename, "r")
content = file.read()
print(content)
file.close()

In [ ]:
with open(filename, 'r') as file:
    lines = file.readlines()[11:-3]
column_headers = ['Parameter','mean(Coef)','std(Coef)','95% Cred ll','95% Cred ul','tStat','Rank','ESS']
asterisks = []

coef_df = pd.DataFrame(columns=column_headers)
for c in range(len(lines)):
    split_line = lines[c].split(' ')
    split_line = list(filter(None, split_line))
    try:
        split_line.remove('|')
    except:
        print('not done')
    matching = [s for s in split_line if "*" in s]
    coef_df.loc[c] = split_line[0],split_line[1],split_line[2],split_line[3],split_line[4],split_line[5],split_line[6],split_line[-1][:-3]
    if len(matching)>0:
        asterisks.append(matching[0])
    if len(matching)==0:
        asterisks.append('')
coef_df['Significance']=asterisks
coef_df

In [ ]:
# if pca:
#     break

In [ ]:
# parcellation_nii = nib.load(inpath+'layered_sbm_l'+str(level)+'.nii.gz')
parcellation_bayes_reg = parcellation.copy()
parcels_to_loop = np.unique(parcellation[parcellation>0]).astype(int)
sig_thresh = ['*','**']
# sig_thresh = ['**']

for p in parcels_to_loop:
    if p in parcellation_bayes_reg:
        subset = coef_df.loc[coef_df['Parameter']=='parcel_'+str(p)]
        
        if len(subset['Significance'])>0:
        
            if subset['Significance'].values[0] in sig_thresh:
                if debug:
                    print('Found something significant: '+str(p))
                parcellation_bayes_reg[parcellation_bayes_reg==p]=float(subset['mean(Coef)'].values[0])
                
            else:
                if debug:
                    print('zero: '+str(p))
                parcellation_bayes_reg[parcellation_bayes_reg==p]=0
        else:
            if debug:
                print('zero: '+str(p))
            parcellation_bayes_reg[parcellation_bayes_reg==p]=0
    else:
        if debug:
            print('zero: '+str(p))
        parcellation_bayes_reg[parcellation_bayes_reg==p]=0

In [ ]:
parcellation_bayes_reg_nii = nib.Nifti1Image(parcellation_bayes_reg,affine=affine)
nib.save(parcellation_bayes_reg_nii,outpath+'parcellation_bayes_reg_nii.nii.gz')
plotting.plot_glass_brain(parcellation_bayes_reg_nii, title='BayesReg Coefficients',colorbar=True,plot_abs=False)
plt.savefig(outpath+'bayesreg.png', dpi=600,bbox_inches='tight')

In [ ]:
parcellation_bayes_reg_pos = parcellation_bayes_reg.copy()
parcellation_bayes_reg_pos[parcellation_bayes_reg_pos<0]=0
parcellation_bayes_reg_nii_pos = nib.Nifti1Image(parcellation_bayes_reg_pos,affine=affine)
nib.save(parcellation_bayes_reg_nii_pos,outpath+'parcellation_bayes_reg_nii_pos.nii.gz')
plotting.plot_glass_brain(parcellation_bayes_reg_nii_pos, title='BayesReg Coefficients (Pos)',colorbar=True,plot_abs=False)
plt.savefig(outpath+'bayesreg_positive_only.png', dpi=600,bbox_inches='tight')

In [ ]:
parcellation_bayes_reg_neg = parcellation_bayes_reg.copy()
parcellation_bayes_reg_neg[parcellation_bayes_reg_neg>0]=0
parcellation_bayes_reg_nii_neg = nib.Nifti1Image(parcellation_bayes_reg_neg,affine=affine)
nib.save(parcellation_bayes_reg_nii_neg,outpath+'parcellation_bayes_reg_nii_neg.nii.gz')
plotting.plot_glass_brain(parcellation_bayes_reg_nii_neg, title='BayesReg Coefficients (Neg)',colorbar=True,plot_abs=False)
plt.savefig(outpath+'bayesreg_negative_only.png', dpi=600,bbox_inches='tight')

In [ ]:
from nilearn import surface
from nilearn import datasets

# stat_img = datasets.load_sample_motor_activation_image()
stat_img = outpath+'parcellation_bayes_reg_nii_pos.nii.gz'

big_fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
big_texture = surface.vol_to_surf(stat_img, big_fsaverage.pial_left)

plotting.plot_surf_stat_map(big_fsaverage.infl_left,
                            big_texture, hemi='left', colorbar=True,
                            title='BayesReg - Regional damage ~ lower Stroop score',
                            threshold=0.001, bg_map=big_fsaverage.sulc_left,
                           )

plt.savefig(outpath+'bayesreg_pos_3d.png', dpi=600,bbox_inches='tight')
plt.show()

In [ ]:
plotting.plot_img_on_surf(stat_img,
                          views=['lateral', 'medial'],
                          hemispheres=['left'],
                          colorbar=True,title='BayesReg - Regional damage ~ lower Stroop score',
                            threshold=0.001)
plotting.show()

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage()
stat_img = outpath+'parcellation_bayes_reg_nii_pos.nii.gz'

texture = surface.vol_to_surf(stat_img, fsaverage.pial_left)

destrieux_atlas = datasets.fetch_atlas_surf_destrieux()
parcellation = destrieux_atlas['map_left']

# these are the regions we want to outline
regions_dict = {b'G_postcentral': 'Postcentral gyrus',
                b'G_precentral': 'Precentral gyrus'}

# get indices in atlas for these labels
regions_indices = [
    np.where(np.array(destrieux_atlas['labels']) == region)[0][0]
    for region in regions_dict
]

labels = list(regions_dict.values())

figure = plotting.plot_surf_stat_map(fsaverage.infl_left,
                                     texture, hemi='left',
                                     title='BayesReg - Regional damage ~ lower Stroop score',
                                     colorbar=True, threshold=.0001,
                                     bg_map=fsaverage.sulc_left)

plotting.plot_surf_contours(fsaverage.infl_left, parcellation, labels=labels,
                            levels=regions_indices, figure=figure,
                            legend=True,
                            colors=['g', 'k'])

plt.savefig(outpath+'bayesreg_pos_3d_with_labels.png', dpi=600,bbox_inches='tight')
plotting.show()